In [45]:
# Загружаем библиотеки
import cv2 # Компьютерное зрение
import numpy as np # Математика
import matplotlib.pyplot as plt # Графики
from sklearn.metrics import accuracy_score # Метрика качества модели
from sklearn.model_selection import train_test_split # Разделение выборки на обучающую и тестовую
from sklearn.svm import SVC # Метод опорных векторов

#### Загрузка данных

In [46]:
# Обучающая выборка
train = np.loadtxt('train.csv', delimiter = ',', skiprows = 1)
# Тестовая выборка
test = np.loadtxt('test.csv', delimiter = ',', skiprows = 1)

In [47]:
# Сохраняем разметку обучающей выборки в отдельную переменную
train_label = train[:, 0]

In [48]:
# Создаём массив с полными картинками 28х28
train_img_full = np.resize(train[:, 1:], (train.shape[0], 28, 28))
# Создаём многомерный массив с размерностями:
# - количество исходных картинок
# - количество фрагментов каждой картинки
# - количество пикселей по горизонтали и вертикали в каждом фрагменте
train_img = np.zeros((train_img_full.shape[0], 4, 14, 14))

In [49]:
# Разбиваем исходные картинки 28х28 на четверти (14х14)
for i in range(train_img_full.shape[0]):
    train_img[i][0] = train_img_full[i, :14, :14]
    train_img[i][1] = train_img_full[i, :14, 14:]    
    train_img[i][2] = train_img_full[i, 14:, :14]          
    train_img[i][3] = train_img_full[i, 14:, 14:]            

In [50]:
test_img_full = np.resize(test, (test.shape[0], 28, 28))
test_img = np.zeros((test_img_full.shape[0], 4, 14, 14))
for i in range(test_img_full.shape[0]):
    test_img[i][0] = test_img_full[i, :14, :14]
    test_img[i][1] = test_img_full[i, :14, 14:]    
    test_img[i][2] = test_img_full[i, 14:, :14]          
    test_img[i][3] = test_img_full[i, 14:, 14:]  

#### Вычисляем X и Y составляющие градиента с помощью оператора Собеля

In [51]:
train_sobel_x = np.zeros_like(train_img)
train_sobel_y = np.zeros_like(train_img)
for i in range(train_img.shape[0]):
    for j in range(train_img.shape[1]):
        train_sobel_x[i][j] = cv2.Sobel(train_img[i][j], cv2.CV_64F, dx=1, dy=0, ksize=3)
        train_sobel_y[i][j] = cv2.Sobel(train_img[i][j], cv2.CV_64F, dx=0, dy=1, ksize=3)

In [52]:
test_sobel_x = np.zeros_like(test_img)
test_sobel_y = np.zeros_like(test_img)
for i in range(test_img.shape[0]):
    for j in range(test_img.shape[1]):
        test_sobel_x[i][j] = cv2.Sobel(test_img[i][j], cv2.CV_64F, dx=1, dy=0, ksize=3)
        test_sobel_y[i][j] = cv2.Sobel(test_img[i][j], cv2.CV_64F, dx=0, dy=1, ksize=3)

#### Вычисляем угол и длину вектора градиента

In [64]:
train_g = np.zeros_like(train_img)
train_theta = np.zeros_like(train_img)
for i in range(train_img.shape[0]):
    for j in range(train_img.shape[1]):
        train_g[i][j], train_theta[i][j] = cv2.cartToPolar(train_sobel_x[i][j], train_sobel_y[i][j])

In [65]:
test_g = np.zeros_like(test_img)
test_theta = np.zeros_like(test_img)
for i in range(test_img.shape[0]):
    for j in range(test_img.shape[1]):
        test_g[i][j], test_theta[i][j] = cv2.cartToPolar(test_sobel_x[i][j], test_sobel_y[i][j])

#### Вычисляем гистограммы градиентов

In [66]:
train_hist = np.zeros((train_img.shape[0], train_img.shape[1], 16))
for i in range(train_img.shape[0]):
    for j in range(train_img.shape[1]):    
        hist, borders = np.histogram(train_theta[i][j],
                                     bins=16,
                                     range=(0., 2. * np.pi),
                                     weights=train_g[i][j])
        train_hist[i][j] = hist

In [67]:
test_hist = np.zeros((test_img.shape[0], test_img.shape[1], 16))
for i in range(test_img.shape[0]):
    for j in range(test_img.shape[1]):    
        hist, borders = np.histogram(test_theta[i][j],
                                     bins=16,
                                     range=(0., 2. * np.pi),
                                     weights=test_g[i][j])
        test_hist[i][j] = hist

#### Разбиваем выборку на обучение и валидацию

In [68]:
y_train, y_val, X_train, X_val = train_test_split(train_label, train_hist, test_size=0.2, random_state=42)

In [69]:
# Компонуем все гистограммы для отдельных фрагментов изображения в одну длинную гистограмму - это вектор признаков
X_train_flatten = np.resize(X_train, (X_train.shape[0], (X_train.shape[1]*X_train.shape[2])))
X_val_flatten = np.resize(X_val, (X_val.shape[0], (X_val.shape[1]*X_val.shape[2])))

#### Для классификации используем метод опорных векторов (SVM)

In [70]:
# Объявляем модель с использованием метода опорных векторов, функция полиномиальная
model = SVC(gamma = 'auto', kernel = 'poly', degree = 3)

In [71]:
# Обучаем модель на обучающей выборке
model.fit(X_train_flatten, y_train)

SVC(gamma='auto', kernel='poly')

#### Оцениваем качество на валидационной выборке

In [72]:
# Валидация
print('Accuracy: %s' % accuracy_score(y_val, model.predict(X_val_flatten)))

Accuracy: 0.9658333333333333


#### Предсказания на тестовыйх данных

In [93]:
test_hist_flatten = np.resize(test_hist, (test_hist.shape[0], (test_hist.shape[1]*test_hist.shape[2])))
pred_test = np.zeros(len(test_img), np.int8)
pred_test = model.predict(test_hist_flatten)

In [95]:
with open('submit.txt', 'w') as dst:
    dst.write('ImageId,Label\n')
    for i, p in enumerate(pred_test.astype(int), 1):
        dst.write('%s,%s\n' % (i, p))

#### Результат на Kaggle

0.96450